In [1]:
import tensorflow as tf 

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import tensorflow as tf
import cv2
import numpy as np

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing
from data.dataloader import DataLoader

In [3]:
def idx2class():
    return dict((v, k) for k, v in Config.pascal_voc_classes.items())


def draw_boxes_on_image(image, boxes, scores, classes):
    num_boxes = boxes.shape[0]
    for i in range(num_boxes):
        class_and_score = str(idx2class()[classes[i]]) + ": " + str(scores[i])
        cv2.rectangle(img=image, pt1=(boxes[i, 0], boxes[i, 1]), pt2=(boxes[i, 2], boxes[i, 3]), color=(255, 0, 0), thickness=2)
        cv2.putText(img=image, text=class_and_score, org=(boxes[i, 0], boxes[i, 1] - 10), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1.5, color=(0, 255, 255), thickness=2)
    return image

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    interpreter.allocate_tensors()
    return interpreter

def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)
    
    input_data = np.array(image, dtype = np.uint8)
    input_details = model.get_input_details()[0]
    output_details = model.get_output_details()[0]
    model.set_tensor(input_details['index'], input_data)
    model.invoke()
    outputs= model.get_tensor(output_details['index'])
    
    print(outputs)
    # 원래 outputs =  소수점 형태
    # quantizaation model =  숫자 너무큼.. output_tensor shape 해결하기
    
    
    
    outputs = tf.constant(outputs)
    
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
    print("-"*50)
    print("boxes")
    print(boxes)
    print("-" * 50)
    print("scores")
    print(scores)
    print("-" * 50)
    print("classes")
    print(classes)
    print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

test_image_dir = "./test_pictures/ck0kfhu4n8q7f0701ixmonyig_jpeg.rf.a3cc5282520b3bac90718bdd5528bd76.jpg"

In [4]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

model_path = "C:/Users/yutan/Desktop/EfficientDet_TensorFlow2/tflite_model_quant.tflite"
efficientdet_lite = load_model(model_path)

image = test_single_picture(picture_dir=test_image_dir, model=efficientdet_lite)

cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
cv2.imshow("detect result", image)
cv2.waitKey(0)

[[[ 0.00297724 -0.03572687  0.         -0.00297724  0.49124447]
  [-0.01786344 -0.0833627   0.02084067 -0.00893172  0.5091079 ]
  [ 0.         -0.04168135 -0.07443098  0.01786344  0.48826724]
  ...
  [-0.00595448  0.03572687  0.05656755  0.03572687  0.49124447]
  [-0.03274963  0.04168135 -0.05061307  0.00595448  0.5388803 ]
  [-0.04763583 -0.04763583 -0.00595448  0.00297724  0.5001762 ]]]
--------------------------------------------------
boxes
[[ 17.467577   0.        52.744617  45.939415]
 [ 37.45707    0.        72.75512   45.939415]
 [ 57.45707    0.        92.75513   45.939415]
 [ 77.45707    0.       112.75513   45.939415]
 [ 97.45706    0.       132.75513   45.939415]
 [117.45706    0.       152.75513   45.939415]
 [137.45706    0.       172.75513   45.939415]
 [157.45709    0.       192.75513   45.939415]
 [177.45709    0.       212.75513   45.939415]
 [197.45709    0.       232.75513   45.939415]
 [217.45709    0.       252.75513   45.939415]
 [237.45709    0.       272.75513 

-1

In [12]:
# 원래 모델
def test_single_picture(picture_dir, model):
    image_array = cv2.imread(picture_dir)
    image = DataLoader.image_preprocess(is_training=False, image_dir=picture_dir)
    image = tf.expand_dims(input=image, axis=0)

    outputs = model(image, training=True)
    print(outputs)
    post_process = PostProcessing()
    boxes, scores, classes = post_process.testing_procedure(outputs, [image_array.shape[0], image_array.shape[1]])
    print("-"*50)
    print("boxes")
    print(boxes)
    print("-" * 50)
    print("scores")
    print(scores)
    print("-" * 50)
    print("classes")
    print(classes)
    print("-" * 50)
    image_with_boxes = draw_boxes_on_image(image_array, boxes.astype(np.int), scores, classes)
    return image_with_boxes

test_image_dir = "./data/fire_smoke/test/JPEGImages/ck0kdhymna0b10721v4wntit8_jpeg.rf.a08e34d04fb672ce6cf8e94e810ec81d.jpg"
if __name__ == '__main__':
    # GPU settings
    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

    efficientdet = EfficientDet()
    efficientdet.load_weights(filepath=Config.save_model_dir + "saved_model")

    image = test_single_picture(picture_dir=test_image_dir, model=efficientdet)

    cv2.namedWindow("detect result", flags=cv2.WINDOW_NORMAL)
    cv2.imshow("detect result", image)
    cv2.waitKey(0)

2021-10-12 22:21:02.683872: W tensorflow/core/common_runtime/bfc_allocator.cc:457] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.00MiB (rounded to 8388608)requested by op FusedBatchNormV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2021-10-12 22:21:02.683963: I tensorflow/core/common_runtime/bfc_allocator.cc:1004] BFCAllocator dump for GPU_0_bfc
2021-10-12 22:21:02.683997: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (256): 	Total Chunks: 93, Chunks in use: 93. 23.2KiB allocated for chunks. 23.2KiB in use in bin. 9.7KiB client-requested in use in bin.
2021-10-12 22:21:02.684020: I tensorflow/core/common_runtime/bfc_allocator.cc:1011] Bin (512): 	Total Chunks: 54, Chunks in use: 54. 33.8KiB allocated for chunks. 33.8KiB in use in bin. 25.3KiB client-requested in use in bin.
2021-10-12 22:21:02.6840

ResourceExhaustedError: OOM when allocating tensor with shape[1,32,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:FusedBatchNormV3]

In [11]:
!nvidia-smi

Tue Oct 12 22:19:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:86:00.0 Off |                  N/A |
| 41%   33C    P8    20W / 280W |   1110MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------